In [1]:
import pandas as pd

In [2]:
metadata=pd.read_csv('movies_metadata.csv')

/var/folders/0x/rj5l5p_d5q5fl64y_t8rc53r0000gn/T/ipykernel_74639/2262726144.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata=pd.read_csv('movies_metadata.csv')


In [4]:
metadata.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [8]:
metadata.id[0]

'862'

In [ ]:
import pandas as pd
import json
import ast

def safe_literal_eval(x):
    try:
        return ast.literal_eval(x)
    except:
        return None

# Read the CSV files with proper settings
print("Reading movies metadata...")
movies_df = pd.read_csv('movies_metadata.csv', low_memory=False)

print("Reading credits data...")
credits_df = pd.read_csv('credits.csv')

# Convert string representations of lists/dicts to actual Python objects
print("\nProcessing JSON-like columns...")
json_columns = ['genres', 'production_companies', 'production_countries', 'spoken_languages']
for col in json_columns:
    movies_df[col] = movies_df[col].apply(safe_literal_eval)

# Process cast and crew columns
credits_df['cast'] = credits_df['cast'].apply(safe_literal_eval)
credits_df['crew'] = credits_df['crew'].apply(safe_literal_eval)

# Extract key information from cast and crew
def extract_cast_info(cast_list):
    if not cast_list:
        return None
    return [{'name': actor.get('name'), 'character': actor.get('character')} 
            for actor in cast_list[:5]]  # Get top 5 cast members

def extract_crew_info(crew_list):
    if not crew_list:
        return None
    directors = [member.get('name') for member in crew_list 
                if member.get('job') == 'Director']
    return directors[0] if directors else None

# Process cast and crew information
credits_df['main_cast'] = credits_df['cast'].apply(extract_cast_info)
credits_df['director'] = credits_df['crew'].apply(extract_crew_info)

# Select only needed columns from credits
credits_processed = credits_df[['id', 'main_cast', 'director']]

# Ensure 'id' columns are of the same type (string)
movies_df['id'] = movies_df['id'].astype(str)
credits_processed['id'] = credits_processed['id'].astype(str)

# Merge the datasets
print("\nMerging datasets...")
merged_df = pd.merge(movies_df, credits_processed, on='id', how='inner')

# Select and reorder important columns
important_columns = [
    'id', 'title', 'original_title', 'release_date', 'runtime',
    'genres', 'vote_average', 'vote_count', 'budget', 'revenue',
    'director', 'main_cast', 'overview'
]

final_df = merged_df[important_columns]

# Save the merged dataset
print("\nSaving merged dataset...")
final_df.to_csv('merged_movies.csv', index=False)

print(f"\nFinal dataset shape: {final_df.shape}")
print("\nSample of the merged data:")
print(final_df.head()) 

In [34]:
merged_movies=pd.read_csv('merged_movies.csv')

In [36]:
merged_movies=merged_movies.drop(columns=['title'])

In [37]:
merged_movies.rename(columns={'original_title': 'title'}, inplace=True)

In [38]:
merged_movies['release_date']=merged_movies['release_date'].apply(lambda x: x.split('-')[0] if pd.notna(x) else None)

In [40]:
merged_movies.rename(columns={'release_year': 'release_date'}, inplace=True)

In [43]:
merged_movies.head(2)

,id,title,release_date,runtime,genres,vote_average,vote_count,budget,revenue,director,main_cast,overview
0,862,Toy Story,1995,81.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",7.7,5415.0,30000000,373554033.0,John Lasseter,"[{'name': 'Tom Hanks', 'character': 'Woody (vo...","Led by Woody, Andy's toys live happily in his ..."
1,8844,Jumanji,1995,104.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",6.9,2413.0,65000000,262797249.0,Joe Johnston,"[{'name': 'Robin Williams', 'character': 'Alan...",When siblings Judy and Peter discover an encha...


In [45]:
merged_movies['genres']=merged_movies['genres'].apply(lambda x: [i['name'] for i in eval(x)])

In [50]:
import ast
def extract_cast_names(cast_list):
    try:
        if pd.isna(cast_list):
            return []
        cast_data = ast.literal_eval(cast_list) if isinstance(cast_list, str) else cast_list
        return [actor['name'] for actor in cast_data]
    except:
        return []

merged_movies['cast'] = merged_movies['main_cast'].apply(extract_cast_names)

In [52]:
import ast
def extract_cast_names(cast_list):
    try:
        if pd.isna(cast_list):
            return []
        cast_data = ast.literal_eval(cast_list) if isinstance(cast_list, str) else cast_list
        return [actor['character'] for actor in cast_data]
    except:
        return []

merged_movies['characters'] = merged_movies['main_cast'].apply(extract_cast_names)

In [54]:
key_words=pd.read_csv('keywords.csv')
key_words.head(2)

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."


In [56]:
key_words['keywords'][0]

"[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]"

In [57]:
merged_movies=pd.merge(merged_movies,key_words,on="id",how='inner')
merged_movies.head(2)

,id,title,release_date,runtime,genres,vote_average,vote_count,budget,revenue,director,main_cast,overview,cast,characters,keywords
0,862,Toy Story,1995,81.0,"[Animation, Comedy, Family]",7.7,5415.0,30000000,373554033.0,John Lasseter,"[{'name': 'Tom Hanks', 'character': 'Woody (vo...","Led by Woody, Andy's toys live happily in his ...","[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[Woody (voice), Buzz Lightyear (voice), Mr. Po...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,Jumanji,1995,104.0,"[Adventure, Fantasy, Family]",6.9,2413.0,65000000,262797249.0,Joe Johnston,"[{'name': 'Robin Williams', 'character': 'Alan...",When siblings Judy and Peter discover an encha...,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[Alan Parrish, Samuel Alan Parrish / Van Pelt,...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."


In [58]:
merged_movies['keywords']=merged_movies['keywords'].apply(lambda x: [i['name'] for i in ast.literal_eval(x)])

In [62]:
links=pd.read_csv('links.csv')

In [63]:
links.head(2)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0


In [64]:
links.rename(columns={'tmdbId':'id'},inplace=True)


In [70]:
merged_movies.drop(columns=['imdbId_x','imdbId_y','movieId_x','movieId_y'],inplace=True)
merged_movies.head(2)

,id,title,release_date,runtime,genres,vote_average,vote_count,budget,revenue,director,main_cast,overview,cast,characters,keywords
0,862,Toy Story,1995,81.0,"[Animation, Comedy, Family]",7.7,5415.0,30000000,373554033.0,John Lasseter,"[{'name': 'Tom Hanks', 'character': 'Woody (vo...","Led by Woody, Andy's toys live happily in his ...","[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[Woody (voice), Buzz Lightyear (voice), Mr. Po...","[jealousy, toy, boy, friendship, friends, riva..."
1,8844,Jumanji,1995,104.0,"[Adventure, Fantasy, Family]",6.9,2413.0,65000000,262797249.0,Joe Johnston,"[{'name': 'Robin Williams', 'character': 'Alan...",When siblings Judy and Peter discover an encha...,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[Alan Parrish, Samuel Alan Parrish / Van Pelt,...","[board game, disappearance, based on children'..."


In [71]:
merged_movies=pd.merge(merged_movies,links,on="id",how='inner')
merged_movies.drop(columns=['movieId'],inplace=True)
merged_movies.head(2)

,id,title,release_date,runtime,genres,vote_average,vote_count,budget,revenue,director,main_cast,overview,cast,characters,keywords,imdbId
0,862,Toy Story,1995,81.0,"[Animation, Comedy, Family]",7.7,5415.0,30000000,373554033.0,John Lasseter,"[{'name': 'Tom Hanks', 'character': 'Woody (vo...","Led by Woody, Andy's toys live happily in his ...","[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[Woody (voice), Buzz Lightyear (voice), Mr. Po...","[jealousy, toy, boy, friendship, friends, riva...",114709
1,8844,Jumanji,1995,104.0,"[Adventure, Fantasy, Family]",6.9,2413.0,65000000,262797249.0,Joe Johnston,"[{'name': 'Robin Williams', 'character': 'Alan...",When siblings Judy and Peter discover an encha...,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[Alan Parrish, Samuel Alan Parrish / Van Pelt,...","[board game, disappearance, based on children'...",113497


In [73]:
merged_movies.rename(columns={'imdbId':'movie_link'},inplace=True)
merged_movies.head(2)

,id,title,release_date,runtime,genres,vote_average,vote_count,budget,revenue,director,main_cast,overview,cast,characters,keywords,movie_link
0,862,Toy Story,1995,81.0,"[Animation, Comedy, Family]",7.7,5415.0,30000000,373554033.0,John Lasseter,"[{'name': 'Tom Hanks', 'character': 'Woody (vo...","Led by Woody, Andy's toys live happily in his ...","[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[Woody (voice), Buzz Lightyear (voice), Mr. Po...","[jealousy, toy, boy, friendship, friends, riva...",114709
1,8844,Jumanji,1995,104.0,"[Adventure, Fantasy, Family]",6.9,2413.0,65000000,262797249.0,Joe Johnston,"[{'name': 'Robin Williams', 'character': 'Alan...",When siblings Judy and Peter discover an encha...,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[Alan Parrish, Samuel Alan Parrish / Van Pelt,...","[board game, disappearance, based on children'...",113497


In [74]:
merged_movies.to_csv('merged_movies.csv',index=False)

In [75]:
merged_movies

,id,title,release_date,runtime,genres,vote_average,vote_count,budget,revenue,director,main_cast,overview,cast,characters,keywords,movie_link
0,862,Toy Story,1995,81.0,"[Animation, Comedy, Family]",7.7,5415.0,30000000,373554033.0,John Lasseter,"[{'name': 'Tom Hanks', 'character': 'Woody (vo...","Led by Woody, Andy's toys live happily in his ...","[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[Woody (voice), Buzz Lightyear (voice), Mr. Po...","[jealousy, toy, boy, friendship, friends, riva...",114709
1,8844,Jumanji,1995,104.0,"[Adventure, Fantasy, Family]",6.9,2413.0,65000000,262797249.0,Joe Johnston,"[{'name': 'Robin Williams', 'character': 'Alan...",When siblings Judy and Peter discover an encha...,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[Alan Parrish, Samuel Alan Parrish / Van Pelt,...","[board game, disappearance, based on children'...",113497
2,15602,Grumpier Old Men,1995,101.0,"[Romance, Comedy]",6.5,92.0,0,0.0,Howard Deutch,"[{'name': 'Walter Matthau', 'character': 'Max ...",A family wedding reignites the ancient feud be...,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[Max Goldman, John Gustafson, Ariel Gustafson,...","[fishing, best friend, duringcreditsstinger, o...",113228
3,31357,Waiting to Exhale,1995,127.0,"[Comedy, Drama, Romance]",6.1,34.0,16000000,81452156.0,Forest Whitaker,"[{'name': 'Whitney Houston', 'character': ""Sav...","Cheated on, mistreated and stepped on, the wom...","[Whitney Houston, Angela Bassett, Loretta Devi...","[Savannah 'Vannah' Jackson, Bernadine 'Bernie'...","[based on novel, interracial relationship, sin...",114885
4,11862,Father of the Bride Part II,1995,106.0,[Comedy],5.7,173.0,0,76578911.0,Charles Shyer,"[{'name': 'Steve Martin', 'character': 'George...",Just when George Banks has recovered from his ...,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[George Banks, Nina Banks, Franck Eggelhoffer,...","[baby, midlife crisis, confidence, aging, daug...",113041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48921,439050,رگ خواب,None,90.0,"[Drama, Family]",4.0,1.0,0,0.0,Hamid Nematollah,"[{'name': 'Leila Hatami', 'character': ''}, {'...",Rising and falling between a man and woman.,"[Leila Hatami, Kourosh Tahami, Elham Korda]","[, , ]",[tragic love],6209470
48922,111109,Siglo ng Pagluluwal,2011,360.0,[Drama],9.0,3.0,0,0.0,Lav Diaz,"[{'name': 'Angel Aquino', 'character': 'Sister...",An artist struggles to finish his work while a...,"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe...","[Sister Angela, Homer, Crazy Woman/Virgin, Ama...","[artist, play, pinoy]",2028550
48923,67758,Betrayal,2003,90.0,"[Action, Drama, Thriller]",3.8,6.0,0,0.0,Mark L. Lester,"[{'name': 'Erika Eleniak', 'character': 'Emily...","When one of her hits goes wrong, a professiona...","[Erika Eleniak, Adam Baldwin, Julie du Page, J...","[Emily Shaw, Det. Mark Winston, Jayne Ferré, A...",[],303758
48924,227506,Satana likuyushchiy,1917,87.0,[],0.0,0.0,0,0.0,Yakov Protazanov,"[{'name': 'Iwan Mosschuchin', 'character': ''}...","In a small town live two brothers, one a minis...","[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa...","[, , , , ]",[],8536
